## [安装python-can](https://python-can.readthedocs.io/en/master/installation.html)

## 视频教程：[手把手教你用Python控制小米微电机](https://www.bilibili.com/video/BV1Qu411P73S)

In [8]:
import os
import sys
# 添加pcan_cybergear库的路径
sys.path.append(os.path.join("..", "cybergear"))

from pcan_cybergear import CANMotorController
import can
import logging
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [13]:
# Connect to the CAN bus with 1 Mbit/s bitrate
bus = can.interface.Bus(bustype="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
motor = CANMotorController(bus, motor_id=127, main_can_id=254)

## Jog
单个参数写入（通信类型 18）

In [4]:
jog_vel = 5  # rad/s
uint_value = motor._float_to_uint(jog_vel, motor.V_MIN, motor.V_MAX, 16)
jog_vel_bytes = motor.format_data(data=[uint_value], format="u16", type="encode")[:2][::-1]

data1 = [0x05, 0x70, 0x00, 0x00, 0x07, 0x01] + jog_vel_bytes
motor.clear_can_rx()
received_msg_data, received_msg_arbitration_id = motor.send_receive_can_message(
    cmd_mode=motor.CmdModes.SINGLE_PARAM_WRITE, data2=motor.MAIN_CAN_ID, data1=data1
)
motor.parse_received_msg(received_msg_data, received_msg_arbitration_id)

2024-05-20 14:34:03,277 - INFO - Sending message with ID 0x1200fe7f, data: [5, 112, 0, 0, 7, 1, 149, 84]
2024-05-20 14:34:03,279 - INFO - Motor CAN ID: 127, pos: 12.50 rad, vel: 0.03 rad/s, torque: -0.00 Nm


(127, 12.5, 0.028839551384756135, -0.0001831082627603564)

## 停止

In [6]:
motor.enable()
motor.send_motor_control_command(torque=1.0, target_angle=3.14, target_velocity=10, Kp=1.0, Kd=1.0)

2024-05-20 14:34:09,288 - INFO - Sending message with ID 0x300fe7f, data: []
2024-05-20 14:34:09,289 - INFO - Motor CAN ID: 127, pos: 12.50 rad, vel: 0.05 rad/s, torque: -0.00 Nm


error: ushort format requires 0 <= number <= (0x7fff * 2 + 1)

In [7]:
motor.disable()

2024-05-20 14:34:13,834 - INFO - Sending message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
2024-05-20 14:34:13,836 - INFO - Motor CAN ID: 127, pos: 12.50 rad, vel: 0.01 rad/s, torque: 0.06 Nm


(127, 12.5, 0.010528725108720494, 0.06024261844815726)

In [ ]:
motor.set_0_pos()

## 读取速度和位置
- 发送参数：单个参数写⼊（通信类型18），写入地址0x7018（电流限制）值为 27A
- 读取返回值：电机反馈数据（通信类型2），Byte0,1: 当前⻆度，Byte2,3:当前⻆速度

In [ ]:
motor.write_single_param(param_name="limit_cur", value=27)

# 位置模式
## 发送电机模式参数写入命令（通信类型 18）
设置 `runmode` 参数为 1
- index(Byte0~1): `run_mode`，0x7005
- value(Byte4~7): 1(位置模式)

In [ ]:
motor.write_single_param("run_mode", value=1)

In [17]:
motor.enable()

2024-05-20 13:21:28,141 - INFO - Sending message with ID 0x300fe7f, data: []
2024-05-20 13:21:28,142 - INFO - Motor CAN ID: 127, pos: 11.94 rad, vel: -0.01 rad/s, torque: -0.00 Nm


(127, 11.941138323033492, -0.011444266422522276, -0.0001831082627603564)

## 最大速度：发送电机模式参数写入命令（通信类型 18）
设置 `limit_spd` 参数为预设最大速度指令
- index(Byte0~1): `limit_spd`, 0x7017
- value(Byte4~7): `float` [0,30]rad/s

In [ ]:
motor.write_single_param("limit_spd", value=10)

## 目标位置：发送电机模式参数写入命令（通信类型 18）
设置 `loc_ref` 参数为预设位置指令
- index(Byte0~1): `loc_ref`, 0x7016
- value(Byte4~7): `float` rad

In [18]:
motor.write_single_param("loc_ref", value=3.14)

2024-05-20 13:21:35,626 - INFO - Sending message with ID 0x1200fe7f, data: [22, 112, 0, 0, 195, 245, 72, 64]
2024-05-20 13:21:35,628 - INFO - Motor CAN ID: 127, pos: 11.93 rad, vel: -0.05 rad/s, torque: 0.03 Nm


(127, 11.930457007705805, -0.04898146028839534, 0.030212863355458808)

In [ ]:
def set_motor_position_control(limit_spd, loc_ref):
    # 设置电机最大速度
    write_single_param(index=0x7017, value=limit_spd, format="f")
    # 设置电机目标位置
    write_single_param(index=0x7016, value=loc_ref, format="f")

In [14]:
import time
# 定义音符到电机参数的映射，为后面的音符分配更高的转速值
note_to_speed = {"C": 6.6, "D": 6.8, "E": 7.3, "F": 8, "G": 9, "A": 10, "B": 13}

# 定义小星星的音符和时长
# 完整定义小星星的音符和时长
notes = ["C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C",
         "G", "G", "F", "F", "E", "E", "D",
         "G", "G", "F", "F", "E", "E", "D",
         "C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C"]

durations = [1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2]  # 以四分音符为单位

x_speed = 1.5 # 速度倍数
x_duration = 0.4 # 间隔倍数，如果小于1，就是减小时间间隔


# 初始化电机的当前位置和方向标志
current_loc_ref = 0
direction_flag = 1  # 1表示正方向，-1表示反方向

motor.disable() #motor.set_0_pos()  # 当前0位
motor.enable()

# 播放小星星
for note, duration in zip(notes, durations):
    duration = duration * x_duration
    
    # 计算电机的转速和目标位置
    limit_spd = note_to_speed[note] * x_speed
    loc_ref = current_loc_ref + (direction_flag * limit_spd * duration * 1.5)

    # 使用电机参数设置函数
    motor.set_motor_position_control(limit_spd=limit_spd, loc_ref=loc_ref)

    # 更新电机的当前位置
    current_loc_ref = loc_ref

    # 反转方向标志
#     direction_flag *= -1

    # 休眠一段时间以模拟音符的时长（这里假设一个四分音符的时长是1秒）
    time.sleep(duration * x_speed)
motor.disable()

PcanCanOperationError: A PCAN Channel has not been initialized yet or the initialization process has failed

In [ ]:
motor.disable()
motor.set_0_pos()

## 速度模式

In [ ]:
motor.write_single_param("run_mode", value=2)

In [ ]:
motor.enable()

In [ ]:
motor.write_single_param("spd_ref", value=5)

In [ ]:
motor.disable()
motor.set_0_pos()